In [ ]:
import numpy as np
import shutil
import cv2
import os
from matplotlib import pyplot as plt

single_folder = "data/SINGLEframes"
double_folder = "data/DOUBLEframes"

current_folder = double_folder

In [ ]:
#Task 1

"""
Firstly, I converted the input color images to the grayscale images, and then used the Canny edge detection 
algorithm to find the edges of the images. Secondly I converted the input images from BGR color space to HSV color
space. The detect_court_color function defines a threshold range for detecting the color of the court and creates 
a mask based on the color range. And then it calculates the number of white pixels in the mask and the total number
of pixels in the image. It will judge whether the court exists based on the ratio of white pixels and a predefined 
threshold. The detect_quadrilateral_in_contours function finds contours in the image. For each contour, 
it calculates the perimeter and approximates the contour to a polygon. If the approximated polygon has four vertices,
it will return True. If these two functions both return True, it indicates that a badminton court is detected.
The results would be output to the folder named "source".
"""
def detect_badminton_court(img):
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray_img, 50, 150, apertureSize=3)
    imgHSV = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    court_color = detect_court_color(imgHSV)
    court_edge, quadrilateral = detect_quadrilateral_in_contours(edges)

    court_present = court_color and court_edge
#     return court_present, edges, quadrilateral
    return court_present


def detect_quadrilateral_in_contours(edge_image):
    contours, _ = cv2.findContours(edge_image, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        perimeter = cv2.arcLength(contour, True)
        approximation = cv2.approxPolyDP(contour, 0.02 * perimeter, True)
        if len(approximation) == 4:
            return True, approximation
    return False, None

def detect_court_color(imgHSV, threshold=0.1):
    lower = np.array([64, 84, 74])
    upper = np.array([100, 145, 255])
    mask = cv2.inRange(imgHSV, lower, upper)
    white_pixels = np.sum(mask == 255)
    total_pixels = mask.shape[0] * mask.shape[1]
    court_ratio = white_pixels / total_pixels
    court_exists = court_ratio > threshold
    return court_exists

images_directory = current_folder
output_directory = "source"

if not os.path.exists(output_directory):
    os.makedirs(output_directory)
    
for filename in os.listdir(images_directory):
    image_path = os.path.join(images_directory, filename)
    image = cv2.imread(image_path)
    if detect_badminton_court(image):
        cv2.imwrite(os.path.join(output_directory, filename), image)

In [ ]:
#Task 2

import cv2
import numpy as np
from matplotlib import pyplot as plt
import os



hsv_param_image_path = 'source/frame635-DOUBLE.jpeg'  # 替换为您的图片路径
hsv_param_image = cv2.imread(hsv_param_image_path)
# hsv_param_image = cv2.cvtColor(hsv_param_image1, cv2.COLOR_BGR2GRAY)


h_min, h_max = 0, 90
s_min, s_max = 0, 72
v_min, v_max = 156, 255
hsv_image_to_modify = cv2.cvtColor(hsv_param_image, cv2.COLOR_BGR2HSV)
mask = cv2.inRange(hsv_image_to_modify, (h_min, s_min, v_min), (h_max, s_max, v_max))
result_image = cv2.bitwise_and(hsv_image_to_modify, hsv_image_to_modify, mask=mask)
result_image_bgr = cv2.cvtColor(result_image, cv2.COLOR_HSV2BGR)
coords = np.array([[186, 720], [1099, 720], [1099, 290], [186, 290]])
# 使用给定的坐标裁剪图像
x_min, y_min = np.min(coords, axis=0)
x_max, y_max = np.max(coords, axis=0)
cropped_image = result_image_bgr[y_min:y_max, x_min:x_max]

# 转换到灰度图像
gray_cropped = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2GRAY)




kernel_size = 3
# 使用均值滤波进行去噪
gray_cropped = cv2.blur(gray_cropped, (kernel_size, kernel_size))

p1 = (183, 337)  # 左上点
p2 = (1101, 337)  # 右上点
p3 = (183, 719)  # 左下点
p4 = (1101, 719)  # 右下点
cv2.rectangle(hsv_param_image, p1, p4, (0, 255, 0), 2)

# 应用Canny边缘检测
edges_cropped = cv2.Canny(gray_cropped, 50, 150)
lines_cropped = cv2.HoughLinesP(edges_cropped, 1, np.pi/180, threshold=50, minLineLength=60, maxLineGap=55)
mid_top_x = (p1[0] + p2[0]) // 2  # 矩形最上方中点的X坐标
left_intercept, right_intercept = None, None

# if lines_cropped is not None:
#     # 从两侧向中间遍历
#     for x in range(p1[0], (p1[0] + p2[0]) // 2):
#         if left_intercept is not None:
#             break
#         for line in lines_cropped:
#             for x1, y1, x2, y2 in line:
#                 if x1 == x2:  # 忽略垂直线
#                     continue
#                 # 计算直线与矩形上边的交点
#                 y_intercept = y1 + (y2 - y1) / (x2 - x1) * (x - x1)
#                 if y_intercept == p1[1]:  # 检查是否在矩形的顶部边缘
#                     left_intercept = x
#                     break
#
#     for x in range(p2[0], (p1[0] + p2[0]) // 2, -1):
#         if right_intercept is not None:
#             break
#         for line in lines_cropped:
#             for x1, y1, x2, y2 in line:
#                 if x1 == x2:  # 忽略垂直线
#                     continue
#                 # 计算直线与矩形上边的交点
#                 y_intercept = y1 + (y2 - y1) / (x2 - x1) * (x - x1)
#                 if y_intercept == p2[1]:  # 检查是否在矩形的顶部边缘
#                     right_intercept = x
#                     break

# # 输出左右两侧的交点坐标
# if left_intercept:
#     print(f"左侧交点坐标：({left_intercept},{p1[1]})")
# else:
#     print("未找到左侧交点")
#
# if right_intercept:
#     print(f"右侧交点坐标：({right_intercept},{p1[1]})")
# else:
#     print("未找到右侧交点")
# # 在原始图像上绘制检测到的线条
for line in lines_cropped:
    x1, y1, x2, y2 = line[0]
    cv2.line(hsv_param_image, (x1 + x_min, y1 + y_min), (x2 + x_min, y2 + y_min), (0, 0, 255), 2)

rightmost_bottom_point = (0, 0)
max_x = 0
max_y = 0

# 检查霍夫变换结果是否为空
if lines_cropped is not None:
    # 遍历每条线，寻找最靠右下角的坐标
    for line in lines_cropped:
        for x1, y1, x2, y2 in line:
            # 检查线段的每个端点
            if x1 > max_x or (x1 == max_x and y1 > max_y):
                max_x, max_y = x1, y1
                rightmost_bottom_point = (x1 + x_min, y1 + y_min)
            if x2 > max_x or (x2 == max_x and y2 > max_y):
                max_x, max_y = x2, y2
                rightmost_bottom_point = (x2 + x_min, y2 + y_min)

# 输出最靠右下角的坐标点
cv2.circle(hsv_param_image, rightmost_bottom_point, 10, (0, 0, 255), -1)  # 紫色的点
print(rightmost_bottom_point)





# 假设 hsv_param_image 是你的输入图像
# rightmost_bottom_point 是你已经找到的最靠右下角的点

# 获取图像的高度和宽度
height, width = hsv_param_image.shape[:2]

# 初始化当前点为最靠右下角的点
current_point = rightmost_bottom_point

# 存储遍历的点
points = [current_point]

# 循环遍历直到到达最上方
while current_point[1] > 0:  # 当前点的y坐标大于0
    x, y = current_point
    found_next_point = False

    # 检查当前点左上方到右上方的点
    for new_x in range(min(x + 1, width - 1), max(x - 2, -1), -1):
        # 检查点是否在图像的右侧边缘
        if hsv_param_image[y - 1, new_x][0] == 255:  # 假设边缘像素为白色
            new_point = (new_x, y - 1)
            points.append(new_point)
            current_point = new_point
            found_next_point = True
            break

    # 如果在当前行找不到下一个点，可能已经到达顶部或遇到断点
    if not found_next_point:
        break

# 用红色圆点标记所有遍历的点
for point in points:
    cv2.circle(hsv_param_image, point, 2, (0, 0, 255), -1)

# 显示结果
# cv2.imshow('Traversed Path', hsv_param_image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()










max_y = 0
min_x = hsv_param_image.shape[1]  # 初始化为图像宽度的最大值
# 遍历检测到的线条以找出最靠左下的坐标点
if lines_cropped is not None:
    for line in lines_cropped:
        for x1, y1, x2, y2 in line:
            # 检查线条的每个端点
            if x1 <= min_x and y1 >= max_y:
                min_x, max_y = x1, y1
            if x2 <= min_x and y2 >= max_y:
                min_x, max_y = x2, y2
# 转换坐标到原图尺寸
leftmost_bottom_point = (min_x + x_min, max_y + y_min)
# 标记最靠左下角的点
cv2.circle(hsv_param_image, leftmost_bottom_point, 10, (255, 0, 255), -1)  # 紫色的点
print(leftmost_bottom_point)



# 使用matplotlib显示最终结果
plt.imshow(cv2.cvtColor(hsv_param_image, cv2.COLOR_BGR2RGB))
plt.axis('off')  # 不显示坐标轴
plt.show()


In [6]:
#Task 3
'''
I used YOLOv5 to detect the athletes by adjusting some parameters such as confidence level. The source images are in the folder named "yolov5-master/data/images". And the results would be output
to the folder named "yolov5-master/runs/detect/exp".
'''
%run yolov5-master/detect.py

detect: weights=yolov5-master/yolov5s.pt, source=yolov5-master/data/images, data=yolov5-master/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.65, iou_thres=0.55, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5-master/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 2024-2-8 Python-3.9.6 torch-2.2.1 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
image 1/225 /Users/zhangzijun/Desktop/ultralytics/yolov5-master/data/images/frame589-DOUBLE.jpeg: 384x640 3 persons, 74.3ms
image 2/225 /Users/zhangzijun/Desktop/ultralytics/yolov5-master/data/images/frame590-DOUBLE.jpeg: 384x640 3 persons, 66.2ms
image 3/225 /Users/zhangzijun/Desktop/ultralytics/yolov5-master/data/images/frame591-DOUBLE.jpeg:

In [ ]:
#Task 4
'''
First, I merged the images filtered from Task 1 into a video. Then, I used YOLOv8 to track and detect the athletes in the video. The result
video would be output to the folder named "source".
'''
import cv2
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO('yolov8n.pt')

# Open the video file
video_path = "source/double"
cap = cv2.VideoCapture(video_path + ".avi")

fourcc = cv2.VideoWriter_fourcc('M', 'P', '4', 'V')
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
out = cv2.VideoWriter(video_path + "_processed.mp4", fourcc, 20.0, (frame_width, frame_height))
while cap.isOpened():
    success, frame = cap.read()

    if success:
        results = model.track(frame, persist=True)

        annotated_frame = results[0].plot()

        out.write(annotated_frame)

    else:
        break

cap.release()
out.release()